# Loading

In [1]:
!pip -q install tensorflow~=2.5.0 numpy~=1.19.5 matplotlib~=3.2.2 tensorflow_hub~=0.12.0 cartopy~=0.19.0

     |████████████████████████████████| 460.3 MB 6.5 kB/s 
     |████████████████████████████████| 14.8 MB 56.3 MB/s 
     |████████████████████████████████| 12.1 MB 28.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 132 kB 65.1 MB/s 
     |████████████████████████████████| 4.0 MB 22.6 MB/s 
     |████████████████████████████████| 1.2 MB 63.4 MB/s 
     |████████████████████████████████| 462 kB 70.2 MB/s 
     |████████████████████████████████| 46 kB 2.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2

In [2]:
# Workaround for cartopy crashes due to the shapely installed by default in
# google colab kernel (https://github.com/anitagraser/movingpandas/issues/81):
!pip uninstall -y shapely
!pip install shapely --no-binary shapely

Found existing installation: Shapely 1.8.2
Uninstalling Shapely-1.8.2:
  Successfully uninstalled Shapely-1.8.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 198 kB 4.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for shapely: filename=Shapely-1.8.2-cp37-cp37m-linux_x86_64.whl size=668422 sha256=013ac575f865894ed2e4e6f9a31b98be2ee81081b56c46f3850015155d7e2ffc
  Stored in directory: /root/.cache/pip/wheels/2f/9e/07/e9e90942b4e31275785d2f7e455607bfe876e53906307f80cd
Successfully built shapely
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


## Imports:

In [3]:
import datetime
import os

!pip install cartopy
import cartopy
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import shapely.geometry as sgeom
import tensorflow as tf
import tensorflow_hub

from google.colab import auth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
matplotlib.rc('animation', html='jshtml')


def plot_animation(field, figsize=None,
                   vmin=0, vmax=10, cmap="jet", **imshow_args):
  fig = plt.figure(figsize=figsize)
  ax = plt.axes()
  ax.set_axis_off()
  plt.close() # Prevents extra axes being plotted below animation
  img = ax.imshow(field[0, ..., 0], vmin=vmin, vmax=vmax, cmap=cmap, **imshow_args)

  def animate(i):
    img.set_data(field[i, ..., 0])
    return (img,)

  return animation.FuncAnimation(
      fig, animate, frames=field.shape[0], interval=24, blit=False)


class ExtendedOSGB(cartopy.crs.OSGB):
  """MET office radar data uses OSGB36 with an extended bounding box."""

  def __init__(self):
    super().__init__(approx=False)

  @property
  def x_limits(self):
    return (-405000, 1320000)

  @property
  def y_limits(self):
    return (-625000, 1550000)

  @property
  def boundary(self):
    x0, x1 = self.x_limits
    y0, y1 = self.y_limits
    return sgeom.LinearRing([(x0, y0), (x0, y1), (x1, y1), (x1, y0), (x0, y0)])


def plot_rows_on_map(rows, field_name="radar_frames", timestep=0, num_rows=None,
                     cbar_label=None, **imshow_kwargs):
  fig = plt.figure(figsize=(10, 10))
  axes = fig.add_subplot(1, 1, 1, projection=ExtendedOSGB())
  if num_rows is None:
    num_rows = next(iter(rows.values())).shape[0]
  for b in range(num_rows):
    extent = (rows["osgb_extent_left"][b].numpy(),
              rows["osgb_extent_right"][b].numpy(),
              rows["osgb_extent_bottom"][b].numpy(),
              rows["osgb_extent_top"][b].numpy())
    im = axes.imshow(rows[field_name][b, timestep, ..., 0].numpy(),
                extent=extent, **imshow_kwargs)

  axes.set_xlim(*axes.projection.x_limits)
  axes.set_ylim(*axes.projection.y_limits)
  axes.set_facecolor("black")
  axes.gridlines(alpha=0.5)
  axes.coastlines(resolution="50m", color="white")
  if cbar_label:
    cbar = fig.colorbar(im)
    cbar.set_label(cbar_label)
  return fig


def plot_animation_on_map(row):
  fig = plt.figure(figsize=(10, 10))
  axes = fig.add_subplot(1, 1, 1, projection=ExtendedOSGB())
  plt.close() # Prevents extra axes being plotted below animation

  axes.gridlines(alpha=0.5)
  axes.coastlines(resolution="50m", color="white")

  extent = (row["osgb_extent_left"].numpy(),
            row["osgb_extent_right"].numpy(),
            row["osgb_extent_bottom"].numpy(),
            row["osgb_extent_top"].numpy())

  img = axes.imshow(
      row["radar_frames"][0, ..., 0].numpy(),
      extent=extent, vmin=0, vmax=15., cmap="jet")

  cbar = fig.colorbar(img)
  cbar.set_label("Precipitation, mm/hr")

  def animate(i):
    return img.set_data(row["radar_frames"][i, ..., 0].numpy()),

  return animation.FuncAnimation(
      fig, animate, frames=row["radar_frames"].shape[0],
      interval=24, blit=False)


def plot_mask_on_map(row):
  fig = plt.figure(figsize=(10, 10))
  axes = fig.add_subplot(1, 1, 1, projection=ExtendedOSGB())
  axes.gridlines(alpha=0.5)
  axes.coastlines(resolution="50m", color="black")

  extent = (row["osgb_extent_left"].numpy(),
            row["osgb_extent_right"].numpy(),
            row["osgb_extent_bottom"].numpy(),
            row["osgb_extent_top"].numpy())

  img = axes.imshow(
      row["radar_mask"][0, ..., 0].numpy(),
      extent=extent, vmin=0, vmax=1, cmap="viridis")

In [5]:
!pip install rasterio

import rasterio
from rasterio.plot import show

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 3.7 MB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# PREPROCESSING

In [7]:
import os
def paths(x1, x2):
  path = r"/content/drive/MyDrive/Colab Notebooks/"

  listOfFolders = os.listdir(path)

  for x in listOfFolders:
    listOfFolders = [i for i in listOfFolders if i.startswith(x1) or i.startswith(x2)]
    
  listOfFolderPaths = [path + y + '/' for y in listOfFolders]

  listOfFolderPaths = sorted(listOfFolderPaths)

  return listOfFolderPaths


cleaned

In [8]:
listOfFolderPathsClean = paths("CLEANEDdata", "CLEANEDdata")
listOfFolderPathsClean

['/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2016-05-23T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2016-06-02T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2016-06-23T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2017-08-29T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2017-08-30T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2017-09-08T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2017-09-09T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2017-09-11T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2017-09-12T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2018-05-29T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2018-05-31T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2019-06-05T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/CLEANEDdata2019-06-06T08:00:00Z/',
 '/content/d

real-time

In [9]:
listOfFolderPathsReal = paths("realTime2021", "realTime2022")
listOfFolderPathsReal

['/content/drive/MyDrive/Colab Notebooks/realTime2021-06-18T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2021-06-20T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2021-06-21T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2021-06-29T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2021-07-13T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2021-07-14T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2021-07-15T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2021-08-21T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2021-08-22T08:00:00Z/',
 '/content/drive/MyDrive/Colab Notebooks/realTime2022-06-05T08:00:00Z/']

In [10]:
def preperingForPred(eventList):
  ListtimestampImagesLists = []

  
  for event in eventList:
    timestampImagesList = []
    filelist = os.listdir(event)
    filelist.sort()
    
    for x in filelist:
      if x.endswith(".tiff"):
        with rasterio.open(event + '/' + x, "r") as  y:
          readTimestamp = y.read()
          timestampImagesList.append(readTimestamp)
        
    timestampImagesList = [tf.convert_to_tensor(x) for x in timestampImagesList]
    timestampImagesList = [tf.expand_dims(x, -1) for x in timestampImagesList]
    timestampImagesList = [tf.cast(x, dtype=tf.float32) for x in timestampImagesList]
    timestampImagesList = tf.concat(timestampImagesList, 0)

    ListtimestampImagesLists.append(timestampImagesList) 
  
  return ListtimestampImagesLists



In [11]:
ListtimestampImagesListsClean = preperingForPred(listOfFolderPathsClean)
ListtimestampImagesListsReal = preperingForPred(listOfFolderPathsReal)

# Joining events - cleaned

In [12]:
ListtimestampImagesListsClean = [tf.cast(i, tf.int32) for i in ListtimestampImagesListsClean]

In [13]:
def event(listTimeStamps, y, z):
  x = tf.concat([listTimeStamps[y], listTimeStamps[z]], 0)
  listTimeStamps[y] = x
  del listTimeStamps[z]
  return listTimeStamps

2017-08-29/30

In [14]:
ListtimestampImagesListsClean = event(ListtimestampImagesListsClean, 3, 4)

2017-09-08/09 

In [15]:
ListtimestampImagesListsClean = event(ListtimestampImagesListsClean, 4, 5)

2017-09-11/12 

In [16]:
ListtimestampImagesListsClean = event(ListtimestampImagesListsClean, 5, 6)

2019-06-05/06 

In [17]:
ListtimestampImagesListsClean = event(ListtimestampImagesListsClean, 8, 9)

Back to float:

In [18]:
ListtimestampImagesListsClean = [tf.cast(i, tf.float32) for i in ListtimestampImagesListsClean]

# Joining events - real-time


In [19]:
len(ListtimestampImagesListsReal)

10

In [20]:
ListtimestampImagesListsReal = [tf.cast(i, tf.int32) for i in ListtimestampImagesListsReal]

In [21]:
ListtimestampImagesListsReal[0].dtype

tf.int32

13-14-15 July 2021 (add 15)

In [ ]:
ListtimestampImagesListsReal = event(ListtimestampImagesListsReal, 4, 5)

In [ ]:
len(ListtimestampImagesListsReal)

9

In [22]:
ListtimestampImagesListsReal = event(ListtimestampImagesListsReal, 4, 5)

In [23]:
len(ListtimestampImagesListsReal)

9

In [24]:
ListtimestampImagesListsReal[4].shape

TensorShape([576, 497, 525, 1])

21-22 July

In [25]:
ListtimestampImagesListsReal = event(ListtimestampImagesListsReal, 5, 6)

In [26]:
len(ListtimestampImagesListsReal)

8

In [27]:
ListtimestampImagesListsReal = [tf.cast(i, tf.float32) for i in ListtimestampImagesListsReal]

# Concatenate Cleaned and Real

In [28]:
ListtimestampImagesList = ListtimestampImagesListsClean + ListtimestampImagesListsReal

In [29]:
len(ListtimestampImagesList)

18

# Correct size for event: [288, 1536, 1280, 1]


FindMax performed before padding to save RAM

In [30]:
def findMax(event):
  maxRain = np.where(event == np.amax(event))[0][0]
  maxAsLast = maxRain - 25
  lastTimestamp = maxAsLast + 22
  selected = event[maxAsLast:lastTimestamp]
  return(selected)

In [31]:
selectedList = [findMax(i) for i in ListtimestampImagesList]

In [32]:
m = 1536 - 497
n = 1280 - 525

In [33]:
tensorList = [tf.constant(i) for i in selectedList]

In [34]:
len(tensorList)

18

In [35]:
tensorList[0].shape

TensorShape([0, 497, 525, 1])

In [36]:
tf.math.floordiv(m, 2)

<tf.Tensor: shape=(), dtype=int32, numpy=519>

In [37]:
tf.math.ceil(m/2)

<tf.Tensor: shape=(), dtype=float32, numpy=520.0>

In [38]:
paddings = tf.constant([[0, 0], [519, 520], [377, 378], [0, 0] ])

In [39]:
tfBigList = [tf.pad(i, paddings, "CONSTANT") for i in tensorList]

In [40]:
shape = tf.reshape(tfBigList[1][0], [1536, 1280])

In [41]:
shape = np.array(shape.shape)
type(shape)

numpy.ndarray

#Load Model from DeepMind

In [42]:
TFHUB_BASE_PATH = "gs://dm-nowcasting-example-data/tfhub_snapshots"

In [43]:
def load_module(input_height, input_width):
  """Load a TF-Hub snapshot of the 'Generative Method' model."""
  hub_module = tensorflow_hub.load(
      os.path.join(TFHUB_BASE_PATH, f"{input_height}x{input_width}"))
  # Note this has loaded a legacy TF1 model for running under TF2 eager mode.
  # This means we need to access the module via the "signatures" attribute. See
  # https://github.com/tensorflow/hub/blob/master/docs/migration_tf2.md#using-lower-level-apis
  # for more information.
  return hub_module.signatures['default']


def predict(module, input_frames, num_samples=1,
            include_input_frames_in_result=False):
  """Make predictions from a TF-Hub snapshot of the 'Generative Method' model.

  Args:
    module: One of the raw TF-Hub modules returned by load_module above.
    input_frames: Shape (T_in,H,W,C), where T_in = 4. Input frames to condition
      the predictions on.
    num_samples: The number of different samples to draw.
    include_input_frames_in_result: If True, will return a total of 22 frames
      along the time axis, the 4 input frames followed by 18 predicted frames.
      Otherwise will only return the 18 predicted frames.

  Returns:
    A tensor of shape (num_samples,T_out,H,W,C), where T_out is either 18 or 22
    as described above.
  """
  input_frames = tf.math.maximum(input_frames, 0.)
  # Add a batch dimension and tile along it to create a copy of the input for
  # each sample:
  input_frames = tf.expand_dims(input_frames, 0)
  input_frames = tf.tile(input_frames, multiples=[num_samples, 1, 1, 1, 1])

  # Sample the latent vector z for each sample:
  _, input_signature = module.structured_input_signature
  z_size = input_signature['z'].shape[1]
  z_samples = tf.random.normal(shape=(num_samples, z_size))

  inputs = {
      "z": z_samples,
      "labels$onehot" : tf.ones(shape=(num_samples, 1)),
      "labels$cond_frames" : input_frames
  }
  samples = module(**inputs)['default']
  if not include_input_frames_in_result:
    # The module returns the input frames alongside its sampled predictions, we
    # slice out just the predictions:
    samples = samples[:, NUM_INPUT_FRAMES:, ...]

  # Take positive values of rainfall only.
  samples = tf.math.maximum(samples, 0.)
  return samples


# Fixed values supported by the snapshotted model.
NUM_INPUT_FRAMES = 4
NUM_TARGET_FRAMES = 18


def extract_input_and_target_frames(radar_frames):
  """Extract input and target frames from a dataset row's radar_frames."""
  # We align our targets to the end of the window, and inputs precede targets.
  input_frames = radar_frames[-NUM_TARGET_FRAMES-NUM_INPUT_FRAMES : -NUM_TARGET_FRAMES]
  target_frames = radar_frames[-NUM_TARGET_FRAMES : ]
  return input_frames, target_frames


def horizontally_concatenate_batch(samples):
  n, t, h, w, c = samples.shape
  # N,T,H,W,C => T,H,N,W,C => T,H,N*W,C
  return tf.reshape(tf.transpose(samples, [1, 2, 0, 3, 4]), [t, h, n*w, c])

In [44]:
matplotlib.rc('animation', html='jshtml')


def plot_animation(field, figsize=None,
                   vmin=0, vmax=10, cmap="jet", **imshow_args):
  fig = plt.figure(figsize=figsize)
  ax = plt.axes()
  ax.set_axis_off()
  plt.close() # Prevents extra axes being plotted below animation
  img = ax.imshow(field[0, ..., 0], vmin=vmin, vmax=vmax, cmap=cmap, **imshow_args)

  def animate(i):
    img.set_data(field[i, ..., 0])
    return (img,)

  return animation.FuncAnimation(
      fig, animate, frames=field.shape[0], interval=24, blit=False)


class ExtendedOSGB(cartopy.crs.OSGB):
  """MET office radar data uses OSGB36 with an extended bounding box."""

  def __init__(self):
    super().__init__(approx=False)

  @property
  def x_limits(self):
    return (-405000, 1320000)

  @property
  def y_limits(self):
    return (-625000, 1550000)

  @property
  def boundary(self):
    x0, x1 = self.x_limits
    y0, y1 = self.y_limits
    return sgeom.LinearRing([(x0, y0), (x0, y1), (x1, y1), (x1, y0), (x0, y0)])


def plot_rows_on_map(rows, field_name="radar_frames", timestep=0, num_rows=None,
                     cbar_label=None, **imshow_kwargs):
  fig = plt.figure(figsize=(10, 10))
  axes = fig.add_subplot(1, 1, 1, projection=ExtendedOSGB())
  if num_rows is None:
    num_rows = next(iter(rows.values())).shape[0]
  for b in range(num_rows):
    extent = (rows["osgb_extent_left"][b].numpy(),
              rows["osgb_extent_right"][b].numpy(),
              rows["osgb_extent_bottom"][b].numpy(),
              rows["osgb_extent_top"][b].numpy())
    im = axes.imshow(rows[field_name][b, timestep, ..., 0].numpy(),
                extent=extent, **imshow_kwargs)

  axes.set_xlim(*axes.projection.x_limits)
  axes.set_ylim(*axes.projection.y_limits)
  axes.set_facecolor("black")
  axes.gridlines(alpha=0.5)
  axes.coastlines(resolution="50m", color="white")
  if cbar_label:
    cbar = fig.colorbar(im)
    cbar.set_label(cbar_label)
  return fig


def plot_animation_on_map(row):
  fig = plt.figure(figsize=(10, 10))
  axes = fig.add_subplot(1, 1, 1, projection=ExtendedOSGB())
  plt.close() # Prevents extra axes being plotted below animation

  axes.gridlines(alpha=0.5)
  axes.coastlines(resolution="50m", color="white")

  extent = (row["osgb_extent_left"].numpy(),
            row["osgb_extent_right"].numpy(),
            row["osgb_extent_bottom"].numpy(),
            row["osgb_extent_top"].numpy())

  img = axes.imshow(
      row["radar_frames"][0, ..., 0].numpy(),
      extent=extent, vmin=0, vmax=15., cmap="jet")

  cbar = fig.colorbar(img)
  cbar.set_label("Precipitation, mm/hr")

  def animate(i):
    return img.set_data(row["radar_frames"][i, ..., 0].numpy()),

  return animation.FuncAnimation(
      fig, animate, frames=row["radar_frames"].shape[0],
      interval=24, blit=False)


def plot_mask_on_map(row):
  fig = plt.figure(figsize=(10, 10))
  axes = fig.add_subplot(1, 1, 1, projection=ExtendedOSGB())
  axes.gridlines(alpha=0.5)
  axes.coastlines(resolution="50m", color="black")

  extent = (row["osgb_extent_left"].numpy(),
            row["osgb_extent_right"].numpy(),
            row["osgb_extent_bottom"].numpy(),
            row["osgb_extent_top"].numpy())

  img = axes.imshow(
      row["radar_mask"][0, ..., 0].numpy(),
      extent=extent, vmin=0, vmax=1, cmap="viridis")

In [45]:
module = load_module(1536, 1280)

# Forecasting

In [46]:
def predictRainfall(eventSelected, name):
  num_samples = 1
  input_frames, target_frames  = extract_input_and_target_frames(eventSelected)
  samples = predict(module, input_frames, num_samples=num_samples, include_input_frames_in_result=False)
  
  return(samples, target_frames)

If shape different than [1, 18, 256, 256, 1] then remove from selectedListReal

In [47]:
tfBigList[1].shape

TensorShape([22, 1536, 1280, 1])

In [48]:
tfBigListInd = [i.shape.as_list() == [22, 1536, 1280, 1] for i in tfBigList]

In [49]:
selectedList = [b for a, b in zip(tfBigListInd, tfBigList) if a == True]

In [50]:
len(selectedList)

14

In [51]:
eventDates = ['2016-05-23', '2016-06-02', '2016-06-23', '2017-08-29,30', '2017-09-08,09', '2017-09-11,12', '2018-05-29', '2018-05-31', '2019-06-05,06', '2020-08-11', '2021-06-18', '2021-06-20', '2021-06-21', '2021-06-29', '2021-07-13,14,15', '2021-08-21,22', '2022-06-05']

In [52]:
len(eventDates)

17

Dates of events which stayed (22 timestamps all included in downloaded data)

In [53]:
selectedDates = [b for a, b in zip(tfBigListInd, eventDates) if a == True]
selectedDates

['2016-06-02',
 '2016-06-23',
 '2017-08-29,30',
 '2017-09-11,12',
 '2018-05-29',
 '2019-06-05,06',
 '2020-08-11',
 '2021-06-18',
 '2021-06-20',
 '2021-06-21',
 '2021-06-29',
 '2021-08-21,22',
 '2022-06-05']

In [54]:
predictions = [predictRainfall(i, '_real') for i in selectedList]

In [55]:
listSampTarg = list(zip(* predictions))
samples = listSampTarg[0]
targer_frames = listSampTarg[1]
targer_frames = [tf.reshape(tf.convert_to_tensor(i), [1, 18, 1536, 1280, 1]) for i in  targer_frames]

In [56]:
targer_frames[12].shape

TensorShape([1, 18, 1536, 1280, 1])

In [57]:
selectedDates[5]

'2019-06-05,06'

# Back to the original shape - in case it's needed

In [58]:
targer_frames[5].shape

TensorShape([1, 18, 1536, 1280, 1])

In [59]:
depad_target = targer_frames[5][:, :, 519:1016, 376:901, :]
depad_samp = samples[5][:, :, 519:1016, 376:901, :]

In [60]:
depad_samp.shape

TensorShape([1, 18, 497, 525, 1])

In [61]:
num_samples = 1
plot_animation(horizontally_concatenate_batch(depad_target), figsize=(4*num_samples, 4))

In [62]:
num_samples = 1
plot_animation(horizontally_concatenate_batch(depad_samp), figsize=(4*num_samples, 4))

Plot samples (prediction)

In [ ]:
num_samples = 1
plot_animation(horizontally_concatenate_batch(samples[5]), figsize=(4*num_samples, 4))

Plot target (observed)

In [ ]:
num_samples = 1
plot_animation(horizontally_concatenate_batch(targer_frames[5]), figsize=(4*num_samples, 4))

# Evaluation Metrices

In [63]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()

In [64]:
targer_frames[0][0].shape

TensorShape([18, 1536, 1280, 1])

In [65]:
len(targer_frames[0][0])

18

In [66]:
for i in targer_frames[0][0]:
  print (i.shape)


(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)
(1536, 1280, 1)


In [67]:
targetFlattenedFullEvent = targer_frames[0][0][0].numpy().flatten()
num_of_above_t = np.count_nonzero(targetFlattenedFullEvent >= 5)
num_of_above_t

42

In [68]:
def MSEwithThreshold(eventsListSamp, eventsListTarg, threshold):
  MSE = dict()


  targetFlattenedFullEvent = eventsListTarg[0].numpy().flatten()
  if threshold == 0:
    num_of_above_t = np.count_nonzero(targetFlattenedFullEvent)
  else:
    num_of_above_t = np.count_nonzero(targetFlattenedFullEvent >= threshold)

  ######
  for i, tens in enumerate(eventsListTarg[0]):
    targetFlattened = tens.numpy().flatten()

    # What to do if no cell in the timestamp meets the requirement??
    if threshold == 0:
      num_of_above_thr_i = np.count_nonzero(targetFlattened)
    else:
      num_of_above_thr_i = np.count_nonzero(targetFlattened >= threshold)
    
    if num_of_above_thr_i == 0:
      #drop timestamp
      eventsListTarg = eventsListTarg[:, ::i, :, :, :]
    if i < len(eventsListTarg[0]):
      continue
  ######

  if len(eventsListTarg[0]) > 0:
    for i in range(len(eventsListTarg[0])):
    #loop over new eventsListTarg
      targetFlattened = eventsListTarg[0][i].numpy().flatten()
      samplesFlattened = eventsListSamp[0][i].numpy().flatten()
      #drop cells below threshold
      if threshold == 0:
        indices = np.where(targetFlattened > threshold)
      else:
        indices = np.where(targetFlattened >= threshold)
      rainOnlyTarget = targetFlattened[indices]
      rainOnlySamples = samplesFlattened[indices]
      MSE[i * 5] = mean_squared_error(rainOnlyTarget, rainOnlySamples)
  else: 
    print("No cells over threshold in the event")


  return(MSE, num_of_above_t, len(eventsListTarg[0]))

#add information which timestamps are missing

In [69]:
MSEwithThreshold(samples[0], targer_frames[0], 1.2)

({0: 4.200557,
  5: 7.8626065,
  10: 6.8815737,
  15: 6.818056,
  20: 7.157749,
  25: 7.0425577,
  30: 9.266829,
  35: 8.230773,
  40: 8.376252,
  45: 9.164438,
  50: 7.9261155,
  55: 7.917442,
  60: 8.235504,
  65: 7.66169,
  70: 8.182461,
  75: 6.9553394,
  80: 8.298684,
  85: 9.13655},
 6776,
 18)

In [70]:
thresholds = [0, 1.5, 2.5, 5]

In [71]:
samples = list(samples)
del targer_frames[9]
del samples[9]

Calculate MSE

In [72]:
MSElist = []
for t in thresholds:
  MSEfull = [MSEwithThreshold(i, j, t) for (i, j) in zip(samples, targer_frames)]
  MSElist.append(MSEfull)

ValueError: ignored

In [ ]:
MSElistReorganized = [[i,j,k,l] for (i, j,k,l) in zip(MSElist[0], MSElist[1], MSElist[2], MSElist[3])]

In [ ]:
MSEdicts = []
MSEnumAboveThres = []
for x in MSElistReorganized:
  MSElist2 = list(zip(* x))
  MSEdict = list(MSElist2[0])
  MSEnumAboveT = list(MSElist[1])
  
  MSEdicts.append(MSEdict)
  MSEnumAboveThres.append(MSEnumAboveT)

Save MSE plots to a folder in Google Disk

In [ ]:
# t0 = MSElist[0]
# t1_5 = MSElist[1]
# t2_5 = MSElist[2]
# t5 = MSElist[3]

calibrated only

In [ ]:
selectedDates

In [ ]:
len(MSElist)

In [ ]:
def extractConv(listOfEvents):
  calibrated = listOfEvents[:7]
  return calibrated

From selected[0][0][0] and from selected[1][0][0] extract only values from dicts

In [ ]:
type(selected[0][0][0])

In [ ]:
len(selected[0][0][0])

In [ ]:
import numpy as np

In [ ]:
def Means(input):  
  t0 = [i for i in input[0][0][0].values()]
  t1_5 = [i for i in input[1][0][0].values()]
  t2_5 = [i for i in input[2][0][0].values()]
  t5 = [i for i in input[3][0][0].values()]
  
  t0Conv = np.mean(t0[-3])
  t0Mix = np.mean(t0[:4])

  t1_5Conv = np.mean(t1_5[-3])
  t1_5Mix = np.mean(t1_5[:4])

  t2_5Conv = np.mean(t2_5[-3])
  t2_5Mix = np.mean(t2_5[:4])

  t5Conv = np.mean(t5[-3])
  t5Mix = np.mean(t5[:4])

  
  return(t0Conv, t0Mix, t1_5Conv, t1_5Mix, t2_5Conv, t2_5Mix, t5Conv, t5Mix)

In [ ]:
means = Means(selected)

In [ ]:
means

In [ ]:
datesConv = sel2[:-3]
datesMixed = sel2[:4]

In [ ]:
mse = MSEdict[0][:7]
mseConv = mse[:-3]
mseMixed = mse[:4]
myConv = [i for i in mseConv[1].values()]
myMixed = [i for i in mseMixed[0].values()]


In [ ]:
len(MSEdict)

In [73]:
min(myConv)

NameError: ignored

In [74]:
selectedDates

['2016-06-02',
 '2016-06-23',
 '2017-08-29,30',
 '2017-09-11,12',
 '2018-05-29',
 '2019-06-05,06',
 '2020-08-11',
 '2021-06-18',
 '2021-06-20',
 '2021-06-21',
 '2021-06-29',
 '2021-08-21,22',
 '2022-06-05']

In [ ]:
del selectedDates[9]

In [ ]:
import seaborn as sns

def plottingMSE(date, listOfDicts, listOfAboveT):
  sns.set_style("whitegrid")
  fig, axes = plt.subplots(1, 4, sharex=True, figsize=(25, 7))
  fig.suptitle(date, fontsize = 15)
  
  MSE_plot = sns.lineplot(data = listOfDicts[0], color="green", ax=axes[0])
  MSE_plot.set(title = "Threshold: 0 \nNr of cells:"+ str(listOfAboveT[0]), xlabel = "time (minutes)", ylabel = "MSE")

  MSE_plot = sns.lineplot(data = listOfDicts[1], color="orange", ax=axes[1])
  MSE_plot.set(title = "Threshold: 1.5 \nNr of cells:"+ str(listOfAboveT[1]), xlabel = "time (minutes)", ylabel = "MSE")
  
  MSE_plot = sns.lineplot(data = listOfDicts[2], color="blue", ax=axes[2])
  MSE_plot.set(title = "Threshold: 2.5 \nNr of cells:"+ str(listOfAboveT[2]), xlabel = "time (minutes)", ylabel = "MSE")
  
  MSE_plot = sns.lineplot(data = listOfDicts[3], color="red", ax=axes[3])
  MSE_plot.set(title = "Threshold: 5 \nNr of cells:"+ str(listOfAboveT[3]), xlabel = "time (minutes)", ylabel = "MSE")
  #MSE_plot.set(title = "Threshold(in milimiters of rain): 5 \nnumber of cells above threshold:"+ str(listOfAboveT[3]), xlabel = "time (minutes)", ylabel = "MSE")
  
  plt.savefig('/content/drive/MyDrive/Colab Notebooks/MSE PLOTS_NEW/' + date + '.png')

In [ ]:
[plottingMSE(i, j, k) for (i, j, k) in zip(selectedDates, MSEdicts, MSEnumAboveThres)]

Print CSI do csv file

Here 0 milimiters of rain is excluded because then it is too easy to classify (mostly 0 ).
t is set to 2.5 beause it is the first level of warning - if classified correctly model can show if there's a reason to be concerned. If the threshold was to high then again it would be to easy to classify.
In DeepMinds research t is lower than 2 mm h-1).
But in their case it seems like CSI goes the other way around: t=2 about 0.5 (same) but the higher threshold the lower their value and in my case it's another way around.

CSI is agregated per event, not timestamp (not in every timestamp 2.5 and above can be found I think).

In [ ]:
import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
# def calculate_CSI(target, prediction, t):
  
#   targetFlattened = target.numpy().flatten()
#   samplesFlattened = prediction.numpy().flatten()
#   indices = np.where(targetFlattened > 0)

#   rainOnlyTarget = targetFlattened[indices]
#   rainOnlySamples = samplesFlattened[indices]


#   pred = (np.array(rainOnlyTarget) < t).astype(int)
#   true = (np.array(rainOnlySamples) < t).astype(int)
#   tn, fp, fn, tp = confusion_matrix(pred, true).ravel()

#   return tp / (tp + fn + fp)

Try to put MSE function just replacing the MSE function with this calculation

In [ ]:
# def calculate_CSI(target, prediction, t):
#   CSIperTimestamp = []

#   targetFlattenedFullEvent = target[0].numpy().flatten()
#   num_of_above_t = np.count_nonzero(targetFlattenedFullEvent)

#   ######
#   for i, tens in enumerate(target[0]):
#     targetFlattened = tens.numpy().flatten()

#     num_of_above_thr_i = np.count_nonzero(targetFlattened)
  
#     if num_of_above_thr_i == 0:
#       #drop timestamp
#       target = target[:, ::i, :, :, :]
#     if i < len(target[0]):
#       continue
#   ######

#   if len(target[0]) > 0:
#     for i in range(len(target[0])):
#     #loop over new eventsListTarg
#       targetFlattened = target[0][i].numpy().flatten()
#       samplesFlattened = prediction[0][i].numpy().flatten()
#       #drop cells below threshold
#       indices = np.where(targetFlattened > 0)

#       rainOnlyTarget = targetFlattened[indices]
#       rainOnlySamples = samplesFlattened[indices]
      
#       pred = (np.array(rainOnlyTarget) < t).astype(int)
#       true = (np.array(rainOnlySamples) < t).astype(int)
#       tn, fp, fn, tp = confusion_matrix(pred, true).ravel()
      
#       csi = tp / (tp + fn + fp)
      
#       CSIperTimestamp.append(csi)
#   else: 
#     print("No cells over threshold in the event")


#   return(CSIperTimestamp)

# #add information which timestamps are missing

In [ ]:
# def calculate_CSI(target, prediction, t):

#   CSIperTimestamp = []
  
#   for i in range(0, len(target[0])):
#     # eventCsiList = []

#     targetFlattened = target[0][i].numpy().flatten()
#     samplesFlattened = prediction[0][i].numpy().flatten()
#     indices = np.where(targetFlattened > 0)
      
#     rainOnlyTarget = targetFlattened[indices]
#     rainOnlySamples = samplesFlattened[indices]
      
#     pred = (np.array(rainOnlyTarget) < t).astype(int)
#     true = (np.array(rainOnlySamples) < t).astype(int)
#     tn, fp, fn, tp = confusion_matrix(pred, true).ravel()
      
#     csi = tp / (tp + fn + fp)
#     # eventCsiList.append(csi)

#     CSIperTimestamp.append(csi)
#   return CSIperTimestamp

In [ ]:
def calculate_CSI(target, prediction, t):

  CSIperTimestamp = []
  
  for i in range(0, len(target[0])):
    # eventCsiList = []

    targetFlattened = target[0][i].numpy().flatten()
    samplesFlattened = prediction[0][i].numpy().flatten()
    indices = np.where(targetFlattened > 0)
      
    rainOnlyTarget = targetFlattened[indices]
    rainOnlySamples = samplesFlattened[indices]
      
    pred = (np.array(rainOnlyTarget) < t).astype(int)
    true = (np.array(rainOnlySamples) < t).astype(int)
    tn, fp, fn, tp = confusion_matrix(pred, true).ravel()
      
    csi = tp / (tp + fn + fp)
    # eventCsiList.append(csi)

    CSIperTimestamp.append(csi)
  return CSIperTimestamp

For event 5 (5-6 June 19) with threshold 2.5 and 2
2
t30: 0.53
t60:
t90:


In [ ]:
calculate_CSI(targer_frames[5], samples[5], 2.5)

In [ ]:
calculate_CSI(targer_frames[5], samples[5], 2.5)[5]

In [ ]:
calculate_CSI(targer_frames[5], samples[5], 2.5)[11]

In [ ]:
calculate_CSI(targer_frames[5], samples[5], 2.5)[17]

In [ ]:
calculate_CSI(targer_frames[5], samples[5], 2)[5]

In [ ]:
calculate_CSI(targer_frames[5], samples[5], 2)[11]

In [ ]:
calculate_CSI(targer_frames[5], samples[5], 2)[17]

In [ ]:
# CSI = [calculate_CSI(i, j, 2.5) for (i,j) in zip(targer_frames, samples)]

In [ ]:
# CSI

# CSI per event

In [ ]:
def CSIPerEvent(target, prediction, t):

  targetFlattened = target.numpy().flatten()
  num_of_above_t = np.count_nonzero(targetFlattened)
  
  samplesFlattened = prediction.numpy().flatten()
  indices = np.where(targetFlattened > 0)
      
  rainOnlyTarget = targetFlattened[indices]
  rainOnlySamples = samplesFlattened[indices]
      
  true = (np.array(rainOnlyTarget) < t).astype(int)
  pred = (np.array(rainOnlySamples) < t).astype(int)
  tn, fp, fn, tp = confusion_matrix(pred, true).ravel()

  csi = tp / (tp + fn + fp)
          
    
  return (csi, num_of_above_t)

In [ ]:
CSI = [CSIPerEvent(i, j, 2.5) for (i,j) in zip(targer_frames, samples)]

In [ ]:
CSIlist1 = list(zip(* CSI))
CSIlist = list(CSIlist1[0])

In [ ]:
CSIlist

In [ ]:
selectedDates